# EEG Project Group 12

## Obtaining Data and Performing Preprocessing


### Importing Required Libraries

From here upto and including EEG Feature Extraction was taken from the sample example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as kr
import seaborn as sns
import os

### Import the Dataset

In [32]:
%%time
#Get Current Working directory and append the data relative dir
cwd = os.getcwd()
print(cwd)
Salt = cwd + r"/data/salt"
Sour = cwd + r"/data/sour"

#Hold file locations
filesSour=[];
filesSalt=[]; #add more later


#populate file location arrays
for file in os.listdir(Salt):
    if file.endswith('csv'):
        filesSalt.append(os.path.join(Salt,file))
for file in os.listdir(Sour):
    if file.endswith('csv'):
        filesSour.append(os.path.join(Sour,file))
#Test reading files by changing num

num = 5;
Salt_sample = pd.read_csv(filesSalt[num])
Sour_sample =  pd.read_csv(filesSour[num])
print(Salt_sample)
print(Sour_sample)



/Users/ericl/Desktop/4th_Year/MANU 465/465_EEG_Group12
                     TimeStamp  Delta_TP9  Delta_AF7  Delta_AF8  Delta_TP10  \
0      2024-11-21 02:13:15.650        NaN        NaN        NaN         NaN   
1      2024-11-21 02:13:15.679   1.353651   0.868130    0.00000    0.705829   
2      2024-11-21 02:13:15.680   1.353651   0.868130    0.00000    0.705829   
3      2024-11-21 02:13:15.680   1.353651   0.868130    0.00000    0.705829   
4      2024-11-21 02:13:15.680   1.353651   0.868130    0.00000    0.705829   
...                        ...        ...        ...        ...         ...   
41474  2024-11-21 02:15:57.433   1.713207   0.529574    0.83422    0.860527   
41475  2024-11-21 02:15:57.433   1.713207   0.529574    0.83422    0.860527   
41476  2024-11-21 02:15:57.433   1.713207   0.529574    0.83422    0.860527   
41477  2024-11-21 02:15:57.434   1.713207   0.529574    0.83422    0.860527   
41478  2024-11-21 02:15:57.434   1.713207   0.529574    0.83422    0.860527 

In [28]:
#Mini-Summary of Block
print(f">{len(filesSalt)} files were added from the Spicy category")
print(f">{len(filesSour)} files were added from the Salty category")



>6 files were added from the Spicy category
>6 files were added from the Salty category


### Available Features

In [29]:
print("Features generated by the Muse 2 headband:")
pd.DataFrame(Salt_sample.columns).T

Features generated by the Muse 2 headband:


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,Gyro_X,Gyro_Y,Gyro_Z,HeadBandOn,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements


### Creating the RAW Dataset

In [34]:
%%time
#Extract rows 21-25 from all files,
#these are the only 5 features relevant for use in the EEG_feature_extraction function

#fileSalt --> 1-6
#fileSour --> 1-6

rowsSalty = []
for f in filesSalt:
    df = pd.read_csv(f)  # Read the file once
    rowsSalty.extend(df.iloc[:, [0, 21, 22, 23, 24, 25]].values.tolist())

rowsSour = []
for f in filesSour:
    df = pd.read_csv(f)  # Read the file once
    rowsSour.extend(df.iloc[:, [0, 21, 22, 23, 24, 25]].values.tolist())


CPU times: user 2.09 s, sys: 451 ms, total: 2.54 s
Wall time: 2.65 s


In [35]:
#Convert to DataFrames
data_Spicy = pd.DataFrame(rowsSalty);
original_Spicy = data_Spicy.copy();
data_Spicy

,0,1,2,3,4,5
0,2024-11-21 14:04:16.799,NaN,NaN,NaN,NaN,NaN
1,2024-11-21 14:04:16.829,851.391941,811.904762,801.831502,814.322344,807.069597
2,2024-11-21 14:04:16.829,861.465201,805.860806,809.084249,844.139194,923.113553
3,2024-11-21 14:04:16.829,833.663004,807.875458,805.457875,816.336996,799.010989
4,2024-11-21 14:04:16.830,815.531136,808.681319,804.249084,782.893773,715.201465
...,...,...,...,...,...,...
191587,2024-11-21 02:15:57.433,988.388278,1190.659341,747.838828,947.289377,1060.512821
191588,2024-11-21 02:15:57.433,1005.714286,1630.256410,921.904762,975.091575,654.761905
191589,2024-11-21 02:15:57.433,1042.783883,1631.062271,732.930403,1074.615385,704.725275
191590,2024-11-21 02:15:57.434,1035.531136,1607.692308,658.791209,1066.556777,941.245421


In [36]:
#Convert to DataFrames
data_Salty = pd.DataFrame(rowsSour);
original_Salty = data_Salty.copy();
data_Salty

,0,1,2,3,4,5
0,2024-11-21 13:55:13.012,NaN,NaN,NaN,NaN,NaN
1,2024-11-21 13:55:13.049,283.260073,793.369963,841.318681,106.373626,735.750916
2,2024-11-21 13:55:13.050,293.333333,799.010989,836.483516,101.538462,660.402930
3,2024-11-21 13:55:13.050,341.684982,796.190476,838.498168,155.128205,854.615385
4,2024-11-21 13:55:13.050,329.597070,795.787546,839.304029,144.652015,944.871795
...,...,...,...,...,...,...
228874,2024-11-21 02:36:36.772,823.186813,792.967033,795.787546,887.252747,908.608059
228875,2024-11-21 02:36:36.773,821.575092,797.802198,789.743590,862.673993,869.523810
228876,2024-11-21 02:36:36.773,794.578755,799.010989,795.384615,844.139194,631.391941
228877,2024-11-21 02:36:36.773,776.849817,805.457875,797.399267,850.183150,707.948718


In [ ]:
#quick check of DataFrames
print(f"Spicy data size is: \t{data_Spicy.shape}",f"\nSalty data size is: \t{data_Salty.shape}")

### Remove Empty Rows

In [ ]:
#Remove NaN values
data_Spicy = data_Spicy.dropna()
data_Salty = data_Salty.dropna()

### Convert Datetime Column to Timestamps

In [ ]:
#Required for compatibility with EEG_feature_extraction function
from datetime import datetime

i = 0;
for time in data_Spicy.iloc[:,0]:
    tmstmp = datetime.strptime(str(time),'%Y-%m-%d %H:%M:%S.%f').timestamp()
    data_Spicy.iat[i,0] = (tmstmp);
i=i+1;

i = 0;
for time in data_Salty.iloc[:,0]:
    tmstmp = datetime.strptime(str(time),'%Y-%m-%d %H:%M:%S.%f').timestamp()
    data_Salty.iat[i,0] = (tmstmp);
i=i+1;


In [ ]:
#Quick Check
data_Spicy.head()

### Save Data to File

Alternative STARTING POINT once data collection is finalized. This step was done to bypass having to run the previous section each time which would take a very long time.

In [ ]:
savelocSpicy = cwd + r"\Data\Preprocessed\Spicy.csv"
savelocSalty = cwd + r"\Data\Preprocessed\Salty.csv"

if os.path.exists(savelocSpicy):
    os.remove(savelocSpicy)
if os.path.exists(savelocSalty):
    os.remove(savelocSalty)
data_Spicy.to_csv(savelocSpicy, mode='w', index = False)
data_Salty.to_csv(savelocSalty, mode='w', index = False)




### EEG Feature Generation

In [ ]:
from eegFG import EEG_feature_extraction as FG

#try various combinations of Nsamp and Perio
Nsamp = 50;
Perio = 6;

xSpicy,ySpicy = FG.generate_feature_vectors_from_samples(file_path=savelocSpicy,
                                                         nsamples=Nsamp,
                                                         period=Perio,
                                                         #state=data_Spicy.iloc[:,-1],
                                                         slide_percent=0.05,
                                                         remove_redundant=False,
                                                         cols_to_ignore=None)
xSpicy.shape


In [ ]:
#try various combinations of Nsamp and Perio
Nsamp = 50;
Perio = 5;

xSalty,ySalty = FG.generate_feature_vectors_from_samples(file_path=savelocSalty,
                                                         nsamples=Nsamp,
                                                         period=Perio,
                                                         #state=data_Salty.iloc[:,-1],
                                                         slide_percent=0.06,
                                                         remove_redundant=False,
                                                         cols_to_ignore=None)
xSalty.shape

In [ ]:
#some quick checks
X_Spicy = pd.DataFrame(np.real(xSpicy))
X_Spicy.columns = np.hstack((['Timestamp'],ySpicy))
X_Spicy.describe()

In [ ]:
with pd.option_context('display.max_rows',None,'display.max_columns',None):
    display(pd.DataFrame(pd.DataFrame(X_Spicy).head()))

In [ ]:
X_Salty = pd.DataFrame(np.real(xSalty))
X_Salty.columns = np.hstack((['Timestamp'],ySalty))
X_Salty.describe()

### Analyze the Dataset

### Separate Into Input and Output

### Encoding the Input

### Encoding the Output

### Split into Test and Train

### Feature Scaling

## Random Forest and SVM

### Importing the Models

### Train and Evaluate the Models

## ANN

### Initialize the ANN

### Adding the Input Layer

### Adding the Hidden Layers

### Adding the Output Layer

### Compiling the ANN

### Training the ANN

### Visualization of Validation

### Using the Model to Predict

### Confusion Matrix